# Analizador sintáctico predictivo ascendente 
Kevin Arturo Amaya Osorio

1) Pedir la gramática LL1 y verificar la info


Analizador sintáctico predictivo ascendente
      1) 
1.   Cada parte del input debe ser separado por un espacio blanco.
2.   Las producciones de la gramática deben ser separados por '->'.
3.   El inicio de las producciones debe empezar con S.
4.   Una producción vacía es denotada por ~ 

ie: 
*   A + B -> C
*   S ->  A B
*   C ->  ~ 

In [ ]:
from google.colab import files
from tabulate import tabulate
import copy

def deleteEmptyInput(input):
  arrayInputs = [x for x in input if x != '']
  return arrayInputs;

def splitInput(inputArray): 
  inputFormatted = [deleteEmptyInput(n.split(' ')) for n in inputArray];
  return inputFormatted;

def getInputFile():
  DATA = str(files.upload()['inputFile'])[2:-3];
  INPUT = DATA.split('\\n');
  SPLIT_ARROW_INPUT = [n.split('->') for n in INPUT];
  PRODUCTIONS = [splitInput(n) for n in SPLIT_ARROW_INPUT];
  return PRODUCTIONS;

EMPTY_CHARACTER = '~';
START_CHARACTER = 'S';
PRODUCTIONS = getInputFile();

Saving inputFile to inputFile (15)


Gets the non terminal and terminal tokens:

In [ ]:
def flattenList(list):
  return [item for sublist in list for item in sublist]

def getTerminalTokens(tokens, NON_TERMINAL_TOKENS):
  return list(set(flattenList(tokens)) - set(NON_TERMINAL_TOKENS));

def deleteDuplicateTokens(tokens):
  return list(set(flattenList(tokens)))

def deleteDuplicates(listDuplicates):
  return list(set(listDuplicates));

def getTerminalTokens(productions, NON_TERMINAL_TOKENS):
  TERMINAL_TOKENS = []
  for production in productions:
    for token in production[1]: 
      if token not in NON_TERMINAL_TOKENS: 
        TERMINAL_TOKENS.append(token)
  return deleteDuplicates(TERMINAL_TOKENS);

def getProductionsDictionary(productions, NON_TERMINAL_TOKENS):
  PRODUCTION_DICTIONARY = dict( [token,[]] for token in NON_TERMINAL_TOKENS);
  for production in productions: 
    PRODUCTION_DICTIONARY[production[0][0]].append(production[1]);    
  return PRODUCTION_DICTIONARY;

NON_TERMINAL_TOKENS = deleteDuplicateTokens([n[0] for n in PRODUCTIONS])
TERMINAL_TOKENS = getTerminalTokens(PRODUCTIONS, NON_TERMINAL_TOKENS);
PRODUCTION_DICT = getProductionsDictionary(PRODUCTIONS, NON_TERMINAL_TOKENS);

print('TERMINAL_TOKENS',TERMINAL_TOKENS)
print('NON_TERMINAL_TOKENS',NON_TERMINAL_TOKENS)
print('PRODUCTION_DICT',PRODUCTION_DICT)

TERMINAL_TOKENS ['~', '(', '*', 'int', '+', ')']
NON_TERMINAL_TOKENS ['F', 'T', "E'", 'S', 'E', "T'"]
PRODUCTION_DICT {'F': [['(', 'E', ')'], ['int']], 'T': [['F', "T'"]], "E'": [['+', 'T', "E'"], ['~']], 'S': [['E']], 'E': [['T', "E'"]], "T'": [['*', 'F', "T'"], ['~']]}


Getting a list of first tokens by implementing the algorithm:

In [ ]:
def isTerminal(production, TERMINAL_TOKENS):
  return production[0] in TERMINAL_TOKENS;

def first(NON_TERMINAL_TOKEN, PRODUCTION_DICT, TERMINAL_TOKENS, ORIGINAL_PROD, FIRST_RECURSION):
  FIRST = []
  for production in PRODUCTION_DICT[NON_TERMINAL_TOKEN]:
    if FIRST_RECURSION: 
      ORIGINAL_PROD = production
      
    if( isTerminal(production, TERMINAL_TOKENS)):
      FIRST.append({'token': production[0],'production': ORIGINAL_PROD});
    else:
      FIRST.extend(first(production[0], PRODUCTION_DICT, TERMINAL_TOKENS, ORIGINAL_PROD, False));
  return FIRST;

FIRSTS = dict()
for token in NON_TERMINAL_TOKENS:
  FIRSTS[token] = first(token, PRODUCTION_DICT, TERMINAL_TOKENS, [], True);

FIRSTS_PRODUCTIONS = dict()
for token in NON_TERMINAL_TOKENS:
  FIRSTS_PRODUCTIONS[token] = []
  for production in FIRSTS[token]:
    FIRSTS_PRODUCTIONS[token].insert(0,production['token'])

print('FIRSTS: ');
print(tabulate(FIRSTS_PRODUCTIONS, headers='keys', tablefmt='fancy_grid'))

FIRSTS: 
╒═════╤═════╤══════╤═════╤═════╤══════╕
│ F   │ T   │ E'   │ S   │ E   │ T'   │
╞═════╪═════╪══════╪═════╪═════╪══════╡
│ int │ int │ ~    │ int │ int │ ~    │
├─────┼─────┼──────┼─────┼─────┼──────┤
│ (   │ (   │ +    │ (   │ (   │ *    │
╘═════╧═════╧══════╧═════╧═════╧══════╛


Getting a list of Follows tokens by implementing the algorithm:

In [ ]:
def getNonRecursiveProds(token, PRODUCTIONS):
  UNIQUE_PRODS = []
  for production in PRODUCTIONS:
    if token not in production:
      UNIQUE_PRODS.append(production);
      continue;
    tokenIndex = production[1].index(token);
    if tokenIndex != (len(production[1])-1):
      UNIQUE_PRODS.append(production);
      continue;
  return UNIQUE_PRODS;

def deleteEmptyCharacter(PRODUCTIONS):
  if EMPTY_CHARACTER in PRODUCTIONS:
    PRODUCTIONS.remove(EMPTY_CHARACTER);
  return PRODUCTIONS

def getProductionsToken(token, PRODUCTION_DICT):
  PRODUCTIONS = [];
  for key in PRODUCTION_DICT:
    for production in PRODUCTION_DICT[key]:
      if token in production:
        PRODUCTIONS.append([key, production])
  return PRODUCTIONS


def isTerminal(token, TERMINAL_TOKENS):
  if token in TERMINAL_TOKENS:
    return True;
  return False;

def hasEmptyFirstProduction(token, FIRSTS_PRODUCTIONS):
  if EMPTY_CHARACTER in FIRSTS_PRODUCTIONS[token]:
    return True
  return False;

def follow(NON_TERMINAL_TOKEN, PRODUCTION_DICT, TERMINAL_TOKENS,FIRSTS_PRODUCTIONS):
  FOLLOW = []
  if(NON_TERMINAL_TOKEN == 'S'):
      return ['$'];
  PRODUCTIONS  = getProductionsToken(NON_TERMINAL_TOKEN, PRODUCTION_DICT);
  UNIQUE_PRODS = getNonRecursiveProds(NON_TERMINAL_TOKEN, PRODUCTIONS);
  try:
    for production in UNIQUE_PRODS:
      KEY = production[0];
      VALUE = production[1];
      INDEX = VALUE.index(NON_TERMINAL_TOKEN);
      if INDEX == (len(VALUE)-1):
        FOLLOW.extend(follow(KEY, PRODUCTION_DICT, TERMINAL_TOKENS, FIRSTS_PRODUCTIONS));
        continue;
      NEXT_VALUE = VALUE[INDEX + 1]
      if(isTerminal(NEXT_VALUE, TERMINAL_TOKENS)):
        FOLLOW.append(NEXT_VALUE);
        continue;
      if(hasEmptyFirstProduction(NEXT_VALUE,FIRSTS_PRODUCTIONS)):
        FOLLOW.extend(FIRSTS_PRODUCTIONS[NEXT_VALUE]);
        FOLLOW.extend(follow(KEY, PRODUCTION_DICT, TERMINAL_TOKENS, FIRSTS_PRODUCTIONS));
        continue;
      FOLLOW.extend(FIRSTS_PRODUCTIONS[NEXT_VALUE]);
  except RecursionError as re:
    return []

  return deleteEmptyCharacter(deleteDuplicateTokens(FOLLOW));

FOLLOW_PRODUCTION = dict();

for token in NON_TERMINAL_TOKENS:
  FOLLOW_PRODUCTION[token] = (follow(token, PRODUCTION_DICT, TERMINAL_TOKENS,FIRSTS_PRODUCTIONS))
print('FOLLOW_PRODUCTION: ');
print(tabulate(FOLLOW_PRODUCTION, headers='keys', tablefmt='fancy_grid'))

FOLLOW_PRODUCTION: 
╒═════╤═════╤══════╤═════╤═════╤══════╕
│ F   │ T   │ E'   │ S   │ E   │ T'   │
╞═════╪═════╪══════╪═════╪═════╪══════╡
│ $   │ $   │ $    │ $   │ $   │ $    │
├─────┼─────┼──────┼─────┼─────┼──────┤
│ *   │ +   │ )    │     │ )   │ +    │
├─────┼─────┼──────┼─────┼─────┼──────┤
│ +   │ )   │      │     │     │ )    │
├─────┼─────┼──────┼─────┼─────┼──────┤
│ )   │     │      │     │     │      │
╘═════╧═════╧══════╧═════╧═════╧══════╛


Creating the Parse Table

In [ ]:
def formatProduction(production,token):
  return token +" -> "+" ".join(production)

def addProduction(row, indexRow, NON_TERMINAL_TOKEN, EMPTY_CHARACTER):
  row[indexRow + 1] = NON_TERMINAL_TOKEN + ' -> ' + EMPTY_CHARACTER;

def getFollowParseRow(NON_TERMINAL_TOKEN, TERMINAL_TOKENS, FOLLOW_PRODUCTION, FIRSTS, row):
  for token in TERMINAL_TOKENS:
    TOKEN_FOUND = False;
    for first in FIRSTS[NON_TERMINAL_TOKEN]:
      if token == first['token']:
        if token == EMPTY_CHARACTER :
          for follow in FOLLOW_PRODUCTION[NON_TERMINAL_TOKEN]:
            indexRow = 0;
            if follow in TERMINAL_TOKENS: 
              indexRow = TERMINAL_TOKENS.index(follow);
            else:
              indexRow = TERMINAL_TOKENS.index(EMPTY_CHARACTER)
            row[indexRow + 1] = (NON_TERMINAL_TOKEN + ' -> ' + EMPTY_CHARACTER);
        break;
  return row;


def getFirstParseRow(NON_TERMINAL_TOKEN, TERMINAL_TOKENS, FIRSTS, row):
  for token in TERMINAL_TOKENS:
    TOKEN_FOUND = False;
    for first in FIRSTS[NON_TERMINAL_TOKEN]:
      if token == first['token']:
        row.append(formatProduction(first['production'],NON_TERMINAL_TOKEN))
        TOKEN_FOUND = True;
        break;
    if not TOKEN_FOUND:
        row.append('')
  return row;

def getParseRow(NON_TERMINAL_TOKEN, FIRSTS, TERMINAL_TOKENS, NON_TERMINAL_TOKENS, PRODUCTION_DICT, FOLLOW_PRODUCTION):
  row = [NON_TERMINAL_TOKEN];
  row = getFirstParseRow(NON_TERMINAL_TOKEN, TERMINAL_TOKENS, FIRSTS, row);
  row = getFollowParseRow(NON_TERMINAL_TOKEN, TERMINAL_TOKENS, FOLLOW_PRODUCTION, FIRSTS, row);
  return row

parseTable = []

for token in NON_TERMINAL_TOKENS:
  parseTable.append(getParseRow(token, FIRSTS, TERMINAL_TOKENS, NON_TERMINAL_TOKENS, PRODUCTION_DICT, FOLLOW_PRODUCTION))

PARSE_TABLE_HEADERS_TOKENS = [];
PARSE_TABLE_HEADERS_TOKENS = copy.deepcopy(TERMINAL_TOKENS);
PARSE_TABLE_HEADERS_TOKENS = list(map(lambda x: x.replace('~', '$'), PARSE_TABLE_HEADERS_TOKENS))
PARSE_TABLE_HEADERS_TOKENS.insert(0, ' ')
print('PARSE TABLE')
print(tabulate(parseTable, headers=PARSE_TABLE_HEADERS_TOKENS, tablefmt='fancy_grid'))

PARSE TABLE
╒═════╤═════════╤════════════╤══════════════╤═══════════╤══════════════╤═════════╕
│     │ $       │ (          │ *            │ int       │ +            │ )       │
╞═════╪═════════╪════════════╪══════════════╪═══════════╪══════════════╪═════════╡
│ F   │         │ F -> ( E ) │              │ F -> int  │              │         │
├─────┼─────────┼────────────┼──────────────┼───────────┼──────────────┼─────────┤
│ T   │         │ T -> F T'  │              │ T -> F T' │              │         │
├─────┼─────────┼────────────┼──────────────┼───────────┼──────────────┼─────────┤
│ E'  │ E' -> ~ │            │              │           │ E' -> + T E' │ E' -> ~ │
├─────┼─────────┼────────────┼──────────────┼───────────┼──────────────┼─────────┤
│ S   │         │ S -> E     │              │ S -> E    │              │         │
├─────┼─────────┼────────────┼──────────────┼───────────┼──────────────┼─────────┤
│ E   │         │ E -> T E'  │              │ E -> T E' │              │   

Get input to generate parse tree

In [ ]:
inputWord = "int + int"

Generate the output table.


In [ ]:
pip install treelib

In [ ]:
def getRowParse(TOKEN, PARSE_TABLE):
  for row in PARSE_TABLE:
    if row[0] == TOKEN:
      return row;
  return []  

def getIndexCharacter(token, TERMINAL_TOKENS):
  if token in TERMINAL_TOKENS:
    return TERMINAL_TOKENS.index(token)+1;
  return -1;

def offSetDerivationRow(OUTPUT_TABLE):
    OFFSET_TABLE = copy.deepcopy(OUTPUT_TABLE);
    for index in range(0, len(OUTPUT_TABLE)-1): 
      OFFSET_TABLE[index][2] = OFFSET_TABLE[index+1][2]
    OFFSET_TABLE[len(OFFSET_TABLE)-1][2] = '';
    return OFFSET_TABLE

def getOutputRow(ROW, PARSE_TABLE, TERMINAL_TOKENS):
  STACK, INPUT_WORD, DERIVATION = copy.deepcopy(ROW[0]), copy.deepcopy(ROW[1]), copy.deepcopy(ROW[2]);
  CHARACTER_STACK = STACK[-1]
  if (CHARACTER_STACK == INPUT_WORD[0]):
    STACK.pop();
    DERIVATION = '';
    del INPUT_WORD[0]
  else:
    CHARACTER_STACK = STACK.pop();
    CHARACTER_INPUT = INPUT_WORD[0];
    INDEX_CHARACTER_INPUT = getIndexCharacter(CHARACTER_INPUT, TERMINAL_TOKENS);
    ROW_PARSE_TABLE = getRowParse(CHARACTER_STACK, PARSE_TABLE)
    DERIVATION = ROW_PARSE_TABLE[INDEX_CHARACTER_INPUT]
    PRODUCTION = []
    if(len(DERIVATION)>0):
      PRODUCTION = deleteEmptyInput(DERIVATION.split('->')[1].split(' '));
    STACK.extend(PRODUCTION[::-1]);     
  return [deleteEmptyCharacter(STACK), deleteEmptyCharacter(INPUT_WORD), DERIVATION];

MAX_SIZE_TABLE = 100000000000;
OUTPUT_TABLE_HEADERS = ['Stack', 'Input', 'Derivation']
STACK = ['$','S']
INPUT_WORD = inputWord.split(' ');
INPUT_WORD.append('$')
DERIVATION = ""
OUTPUT_TABLE = []
OUTPUT_ROW = [STACK, INPUT_WORD, DERIVATION];
OUTPUT_TABLE.append(OUTPUT_ROW)

for i in range(0,MAX_SIZE_TABLE):
  OUTPUT_ROW = getOutputRow(OUTPUT_ROW, parseTable, TERMINAL_TOKENS);
  if OUTPUT_ROW[0] == [] and OUTPUT_ROW[1] == []:
    break;
  OUTPUT_TABLE.append(OUTPUT_ROW)

OUTPUT_TABLE = offSetDerivationRow(OUTPUT_TABLE)
print(tabulate(OUTPUT_TABLE, headers=OUTPUT_TABLE_HEADERS, tablefmt='fancy_grid'))

╒══════════════════════════╤══════════════════════════╤══════════════╕
│ Stack                    │ Input                    │ Derivation   │
╞══════════════════════════╪══════════════════════════╪══════════════╡
│ ['$', 'S']               │ ['int', '+', 'int', '$'] │ S -> E       │
├──────────────────────────┼──────────────────────────┼──────────────┤
│ ['$', 'E']               │ ['int', '+', 'int', '$'] │ E -> T E'    │
├──────────────────────────┼──────────────────────────┼──────────────┤
│ ['$', "E'", 'T']         │ ['int', '+', 'int', '$'] │ T -> F T'    │
├──────────────────────────┼──────────────────────────┼──────────────┤
│ ['$', "E'", "T'", 'F']   │ ['int', '+', 'int', '$'] │ F -> int     │
├──────────────────────────┼──────────────────────────┼──────────────┤
│ ['$', "E'", "T'", 'int'] │ ['int', '+', 'int', '$'] │              │
├──────────────────────────┼──────────────────────────┼──────────────┤
│ ['$', "E'", "T'"]        │ ['+', 'int', '$']        │ T' -> ~      │
├─────

Generate Parse Tree

In [ ]:
from treelib import Node, Tree

def getRowParseTree(TOKEN, PARSE_TABLE):
  for row in PARSE_TABLE:
    print(TOKEN)
    if row[0]== TOKEN['value'] :
      return row;
  return []  

def getDerivations(OUTPUT_TABLE):
  COLUMN = []
  for row in OUTPUT_TABLE:
    COLUMN.append(row[2])
  return COLUMN;

def checkEmpty(rule):
  return rule != ''; 

def deleteEmptyCharacterTree(STACK):
  TEMP_STACK = []
  for item in STACK:
    if item['value'] != EMPTY_CHARACTER:
      TEMP_STACK.append(item);
  return TEMP_STACK;

def generateTree(ROW, PARSE_TABLE, TERMINAL_TOKENS, NODE_ROOT, INDEX):
  STACK, INPUT_WORD, DERIVATION = copy.deepcopy(ROW[0]), copy.deepcopy(ROW[1]), copy.deepcopy(ROW[2]);
  TOKEN_STACK = STACK[-1];
  TOKEN_INPUT = INPUT_WORD[0];
  if TOKEN_STACK['value'] == TOKEN_INPUT:
    STACK.pop();
    DERIVATION = '';
    del INPUT_WORD[0]
  #Detect that a character is a production
  else:
    INDEX_TOKEN_INPUT = getIndexCharacter(TOKEN_INPUT, TERMINAL_TOKENS);
    ROW_PARSE_TABLE = getRowParse(TOKEN_STACK['value'], PARSE_TABLE);
    DERIVATION = ROW_PARSE_TABLE[INDEX_TOKEN_INPUT].split('->');
    DERIVATION_GENERATOR = list(filter(checkEmpty,deleteEmptyInput(DERIVATION[0])));
    DERIVATION_PRODUCTION = list(filter(checkEmpty, DERIVATION[1].split(' ')));
    STACK.pop();
    PARENT_NODE = TOKEN_STACK['node'];
    TEMP_STACK = []
    for derProd in DERIVATION_PRODUCTION:
      NODE_TOKEN = tree.create_node(derProd, derProd + str(INDEX), parent=PARENT_NODE.identifier);
      TOKEN_STACK = {'value': derProd, 'node': NODE_TOKEN};
      TEMP_STACK.append(TOKEN_STACK)
    TEMP_STACK = deleteEmptyCharacterTree(TEMP_STACK);
    STACK.extend(TEMP_STACK[::-1]);
  return [STACK, INPUT_WORD, DERIVATION];


tree = Tree()

STACK = [{'value':'$', 'node':None},{'value': 'S', 'node': tree.create_node("S", "S")}]
INPUT_WORD = inputWord.split(' ');
INPUT_WORD.append('$')
DERIVATION = "";
TREE_OUTPUT_ROW = [STACK, INPUT_WORD , DERIVATION];
DERIVATIONS = deleteEmptyInput(getDerivations(OUTPUT_TABLE));
OUTPUT_TABLE_TREE = []
TREE_ARR = []
OUTPUT_THREE_TABLE_HEADERS = ['Stack', 'Input', 'Derivation', 'Tree'];

for j in range(0,14):
  TREE_OUTPUT_ROW = generateTree(TREE_OUTPUT_ROW, parseTable, TERMINAL_TOKENS, None, j);
  if TREE_OUTPUT_ROW[0] == [] and TREE_OUTPUT_ROW[1] == []:
    break;
  OUTPUT_TABLE_TREE.append(TREE_OUTPUT_ROW)

print(tree)


S
└── E
    ├── E'
    │   ├── +
    │   ├── E'
    │   │   └── ~
    │   └── T
    │       ├── F
    │       │   └── int
    │       └── T'
    │           └── ~
    └── T
        ├── F
        │   └── int
        └── T'
            └── ~

